# 2-Autograd: 自动求导

In [1]:
# PyTorch 中所有神经网络的核心是`autograd`包.我们首先简单介绍一下这个包,
# 然后训练我们的第一个神经网络.
# `autograd`包为张量上的所有操作提供了自动求导.
# 它是一个运行时定义的框架,
# 这意味着反向传播是根据你的代码如何运行来定义,并且每次迭代可以不同.

# 张量 Tensor
`torch.Tensor`是包的核心类。如果将其属性`.requires_grad`设置为True，则会开始跟踪其上的所有操作。完成计算后，您可以调用`.backward()`并自动计算所有梯度。此张量的梯度将累积到`.grad`属性中。

要阻止张量跟踪历史记录，可以调用`.detach()`将其从计算历史记录中分离出来，并防止将来的计算被跟踪。

要防止跟踪历史记录（和使用内存），您还可以使用torch.no_grad()包装代码块：在评估模型时，这可能特别有用，因为模型可能具有`requires_grad = True`的可训练参数，但我们不需要梯度。

还有一个类对于autograd实现非常重要 - Function。

Tensor和Function互相连接并构建一个非循环图构建一个完整的计算过程。每个张量都有一个`.grad_fn`属性，该属性引用已创建Tensor的Function（除了用户创建的Tensors  - 它们的`grad_fn`为`None`）。

如果要计算导数，可以在Tensor上调用`.backward()`。如果Tensor是标量（即它包含一个元素数据），则不需要为`backward()`指定任何参数，但是如果它有更多元素，则需要指定一个梯度参数，该参数是匹配形状的张量。

In [2]:
import torch

In [3]:
# 传建一个张量，并设置 requires_grad=True 来跟踪它的计算

In [4]:
x = torch.ones(2,2,requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [5]:
y = x + 2
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

In [6]:
y.grad_fn

In [7]:
x.grad_fn

In [8]:
z = y * y * 3
out = z.mean()
z,out

(tensor([[27., 27.],
         [27., 27.]], grad_fn=<MulBackward0>),
 tensor(27., grad_fn=<MeanBackward1>))

In [9]:
a = torch.randn(2,2)
a = ((a * 3) / (a - 1))

In [10]:
a

tensor([[-37.0705,   1.6267],
        [ -1.9086,  -0.5302]])

In [11]:
a.requires_grad

False

In [12]:
# `.requires\_grad_(...)`就地更改现有的Tensor的`requires_grad`标志。 如果没有给出，输入标志默认为False。

In [13]:
a.requires_grad_(True)
a.requires_grad

True

In [14]:
b = (a*a).sum()

In [16]:
b.grad_fn

## 梯度(Gradients)

现在我们来执行反向传播,`out.backward()`相当于执行`out.backward(torch.tensor(1.))`

In [17]:
out.backward()

In [18]:
x.grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

In [19]:
out

tensor(27., grad_fn=<MeanBackward1>)

In [20]:
x = torch.randn(3,requires_grad=True)
y = x*2

In [21]:
x

tensor([-0.0656, -0.1010, -0.4716], requires_grad=True)

In [22]:
y

tensor([-0.1313, -0.2019, -0.9433], grad_fn=<MulBackward0>)

In [23]:
while y.data.norm() < 1000:
    y = y * 2

In [24]:
y

tensor([ -268.8669,  -413.5849, -1931.8119], grad_fn=<MulBackward0>)

In [25]:
y.data.norm()

tensor(1993.8002)

In [26]:
y =torch.ones(3)

In [27]:
y

tensor([1., 1., 1.])

In [28]:
y.norm

<bound method Tensor.norm of tensor([1., 1., 1.])>

In [29]:
y.data.norm

<bound method Tensor.norm of tensor([1., 1., 1.])>

In [30]:
y.data.norm()

tensor(1.7321)

现在在这种情况下，y不再是标量。 `torch.autograd`无法直接计算完整雅可比行列式，但如果我们只想要雅可比向量积，只需将向量作为参数向后传递：

In [32]:
y = x*2
while y.data.norm() < 1000:
    y = y * 2
v = torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
y.backward(v)
x.grad

tensor([4.0960e+02, 4.0960e+03, 4.0960e-01])

In [33]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False
